<img src="docs/ajccbc.png">
<p></p>

<h1>Introduction: Enriching data</h1>

*by Serge Droz and Eireann Leverett*

In this second lessen we will focus on enriching data using external information. These can be of different types, `http`, `DNS` or pretty much anything else. You should be familiar with basic `REST` APIs, and `DNS`.

**Note**: This Notebook will not work in `Jupyterlite` 

#### License

This material is available under the [Creative Commons BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

This course includes GeoLite2 Data created by MaxMind, available from
[https://www.maxmind.com](https://www.maxmind.com/).

## Initialization 

Please run the cell below by **putting the mouse into the cell** and then **pressing *shift enter***. . First we read in the same data file we used in lesson one. 

In [ ]:
import re
import datetime
import os
import pandas as pd 

parse_log_line = re.compile( r'([0-9a-f:\.]+) - ([^\s]+) \[([^\]]+)] "([^\"]*)" (\d+) (\d+) "([^\"]*)" "([^\"]*)" "([^\"]*)"\s*(\d*)\s*([^\s]*)$'  )
def read_logs( filename ):
    """Read in a ngnix log and split it into fileds.
       filname: the name of the logfile
       Returns: a data frame containing the log"""
    data = []
    with open( filename ) as fp:
        for i, line in enumerate(fp.read().split("\n")):
            if not line:
                continue
            tmp = parse_log_line.findall( line ) 
            if  len ( tmp ): 
                data.append( *tmp )
            else:
                print( "Error parsing line {:2d}".format(i))

    columns= [ "IP", 'User', 'Time', 'Request', 'Status', 'Body Bytes', 'Referer', 'User-Agent', 'Forwarded', 'Dest-Port', 'Host']


    df = pd.DataFrame( data, columns=columns )
    df['Status']     = df['Status'].apply( lambda x: int(x))
    df['Body Bytes'] = df['Body Bytes'].apply( lambda x: int(x))
    df['Dest-Port']  = df['Dest-Port'].apply( lambda x:  int(x) if x else -1  )
    df['Time'] = pd.to_datetime( df['Time'], format="%d/%b/%Y:%H:%M:%S %z" , utc=True )
    df['Time'] = df['Time'].apply( lambda x: x.tz_convert('UTC').tz_localize(None) ) # Convert to UTC
    return df
df = read_logs( 'data/webserver-log-01.txt' )
df.head()

## DNS

Analyzing logfiles very often involves dealing with IP addresses. Thus it's natural to see what domain name belongs to an IP address. 
The most straight forward way is doing a *reverse lookup*. This is easily done in Python.


In [ ]:
import dns.resolver

resolver = dns.resolver.Resolver()

def rlookup( ip ):
    q =  dns.reversename.from_address( ip )
    try:
        return resolver.resolve(q, rdtype='PTR').rrset[0].to_text()
    except:
        return ""
    
%time rlookup( '54.175.213.155')
%time rlookup( '54.175.213.155')

This seems to work. But one thing to keep in mind with network based queries is that they tend to take some time. The magic command `%time` shows you how much. But careful! DNS e.g. may be cached, so a second lookup may be a lot faster. But let's check the situation for DNS with our data. 

In [ ]:
list( range( 0, 100, 10 ) ) 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  
import time

def time_it( n ):
    start = time.time()
    [ rlookup( ip ) for ip in df[:n]['IP'] ]
    return time.time() - start

seconds = [ time_it( i ) for i in range( 20 ) ]
fig, ax = plt.subplots( figsize=(8,5) )

ax.bar( range(len(seconds)), seconds )
ax.set_xlabel("Number of queries")
ax.set_ylabel("Seconds")
ax.grid()

Obviously this does not work very efficiently if we have to perform many queries. 

There simple methods to alleviate the problem:

 - Caching: This works well if we we have have many equal requests
 - Threading: This is a bit more complicated, but works well if individual queries just need time, i.e. the code is mostly waiting for answers. 

Let's create an object that does both. The code is a bit involved, so don't worry if it doesn't immediately make sense. 

Essentially this class provides a function that takes an array of IP addresses and returns a dictionary of IPs pointing to the hostname. Note that there ay be other host name that point to the same IP address.

This object already contains code to do DNS blacklist lookups, we';; come to this later. 

In [ ]:
import threading
import dns.resolver

class FastResolver( object ):
    
    def split(self, a, n):
        """split a list into n sublists"""
        # See https://stackoverflow.com/questions/2130016/splitting-a-list-into-n-parts-of-approximately-equal-length
        k, m = divmod(len(a), n)
        return list( (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)) )

    def __init__( self, n = 10 ):
        """Initialize the object. You can specify how many threads you want run concurently"""
        self.n = n 
        self.resolver = dns.resolver.Resolver()
        self.bl_form_query = lambda ip,bl:'.'.join( ( *reversed(ip.split(".") ), bl ) )
    
    def _r_lookup( self, ips, results, i ):
        """Do an reverse lookup on the ips in the array ips"""
        while ips[i]:
            ip = ips[i].pop()
            try:
                q = dns.reversename.from_address( ip )
                results[ip] = self.resolver.resolve(q, rdtype='PTR').rrset[0].to_text()
            except:
                pass
        
    def reverse_lookup_list(self, ips):
        """Do an reverse lookup on the ips in the array ips by running threads"""
        self.ips  = list( set(ips) )
        sublists = self.split( self.ips, self.n )
        threads = []
        results = {}
        for index in range(self.n):
            x = threading.Thread(target=self._r_lookup, args=( sublists, results, index  ) )
            threads.append(x)
            x.start()
        for thread in threads:
            thread.join()
            
        #return  { k: d[k] for d in results for k in d.keys() }
        return results
    
    def _bl_lookup( self, ips, results, i, bl ):
        """Look up a blacklist"""
        while ips[i]:
            ip = ips[i].pop()
            try:
                results[i][ip] = self.resolver.resolve( self.bl_form_query(ip,bl) , 'A')[0].to_text()
            except:
                pass

    def bl_lookup_list(self,ips,bl):
        """Lookup a an array of ips in a DNS blacklist"""
        self.ips  = list(set(ips))
        sublists = self.split( self.ips, self.n )
        results = {}
        threads = []
        for index in range(self.n):
            x = threading.Thread(target=self._bl_lookup, args=(sublists, results, index, bl ) )
            threads.append(x)
            x.start()

        for thread in threads:
            thread.join()
        return results
        #return { k: d[k] for d in results for k in d.keys() }
    
    
fresolver = FastResolver()
%time r = fresolver.reverse_lookup_list( df[:30]['IP']  )
#%time r = fresolver.bl_lookup_list( df[-1000:] , 'zen.spamhaus.org.'  )
r

In [ ]:
def time_it( n ):
    start = time.time()
    fresolver.reverse_lookup_list( df[:n]['IP']  )
    return time.time() - start

seconds = [ time_it( i ) for i in range( 20 ) ]
fig, ax = plt.subplots( figsize=(8,5) )

ax.bar( range(len(seconds)), seconds )
ax.set_xlabel("Number of queries")
ax.set_ylabel("Seconds")
ax.grid()

This looks much better. This plot may look a bit random. This comes from the fact that DNS itself caches answers. Let's run quickly through the code. For the moment we ignore the functions related to black list lookups (`bl_`)

We implemented this as a class. This is good practice for code that may be reused often. We need two python modules:

 - `dns.resolver` which does the actual DNS lookups
 - `threading` which is a python interface to [threads](https://docs.python.org/3/library/threading.html)
 
The *constructor* takes one optional argument (besides the `self`), the number of threads. Its set to 10 by default. It then sets up resolver instance. As said, we will ignore the blacklist related stuff later. 

The function `reverse_lookup_list` takes an array of ip4 addresses, separates it into `n` sub-lists and sets up the treads, which then execute `_r_lookup`. This functions takes the sub-list of IPs as well as a dictionary, `results`. It i an answer is found, it's added as a key value pair to this dictionary. 

That's it essentially. Threading is not a simple topic, but its beyond this tutorial to explain it, as there are many good explanations out there. 

This class only handles IPv4 addresses. It can easily be extended to IPv6 addresses though. 

## Applying this to the available data

The above methods returns a dictionary with IP, Names key, value pairs. So we create this dict first and then apply it to the dataframe. Despite the treading, this will take a bit of time.

In [ ]:
%time r = fresolver.reverse_lookup_list( df['IP']  )
df['DNS'] = df['IP'].apply(lambda x: r[x] if x in r.keys() else "")

Doing a `value_count` you can see that a lot of IPs don't have a reverse lookup configured. 

In [ ]:
df['DNS'].value_counts()

## Leveraging DNS 

We can now start looking at the returned DNS names. Remember, that in the fist lesson we tried to identify scanners from their User-Agent strings. Some scanners, especially if they scan ports other than 80 or 443, have this info in their DNS names. Let's see if we find such scanner.


In [ ]:
ua_scanner_keywords = [ 'project_patchwatch', 'scaninfo@expanseinc.com', 'project-resonance.com' ,
                            'gdnplus.com', 'censys.io', 'netsystemsresearch.com', 'Nmap Scripting Engine'
                            'internet-measurement.com', 'InternetMeasurement', 'tchelebi.io', 'paloaltonetworks.com', 'security.ipip.net' ] 
dns_scanner_keywords = ['shodan.io', 'research-scanner.com', 'censys-scanner.com', 'shadowserver.org', 'the-shadow-server-foundation', 'internet-measurement.com', 'internet-census.org', 'netsystemsresearch.com', 'leakix.org', 'stretchoid.com', 'recyber.net', 'planetlab', 'criminalip.com', 'quadmetrics.com', 'scanopticon.com', 'onyphe.net', 'crd.bi.zone', 'michael-kipp.com', 'mapletoast.com', 'maaketing.nl', 'intenettl.org', 'iostation.io', 'intrinsec.com', 'security.ipip.net', 'rnd.group-ib.com', 'threatsinkhole.com', 'binaryedge.ninja']

def is_scanner( x ):
    
    tmp = []
    for kw in dns_scanner_keywords:
        if kw in x['DNS']:
            tmp = ['DNS']
            break
    for kw in ua_scanner_keywords:
        if kw in x['User-Agent']:
            tmp.append( "UA" )

    return ','.join(tmp)

df['Scanner'] = df.apply( is_scanner, axis = 1 )

ax = df['Scanner'].value_counts().plot(kind="pie",figsize=(6,6), fontsize=12)
_ = ax.set_ylabel("")

A lot more IPs scan than the User-Agent analysis may make you believe. 

## DNS Blocklists

Often IPs are already known to be malicious. A great source of such info are DNS blocklists (DNSBL), although  the name blocklist is missleading. The decission of what to do with an IP is independent of it being available. 

DNS has the advantageof being fast. DNSBL all seem to use the same mechanisms. For an IPv4 A.B.C.D it works essentially is a A record lookup for D.C.B.A.some.domain, where some.domain is specific for the blacklist used. E.g. for Spamhaus' ZEN list it would be `zen.spamhaus.org.`. The replies are IP addresses, by convention all in `127.0.0.0/8`. Depending on the blacklist different answers are returned indication the abuse. Again for ZEN these are

```
        "127.0.0.2" : "SBL Spamhaus SBL Data",
        "127.0.0.3" : "SBL Spamhaus SBL CSS Data",
        "127.0.0.4" : "XBL CBL Data",
        "127.0.0.9" : "SBL Spamhaus DROP/EDROP Data",
        "127.0.0.10" : "PBL ISP Maintained",
        "127.0.0.11" : "PBL Spamhaus Maintained",
        "127.255.255.254" : 'Error'
```

### Exercise

Write a function that takes an IPv4 address and queries ZEN, returning the result. 


In [ ]:
# Write your answer here and test a few of the addresses in the logs.

The above DNS class has already support for queying DNSBLs.
Ideally a dataframe is updated with the reason as well as the return code. 

## Other sources

There are a variety of others free TI sources, such as MISP instances, OTX, Cymru Whois etc.
Some have their own Python modules others, such as the Cymru Whois service need custom code. 
This really is not much different from what we did so far. 

However many ressources have rate limits, so remember to cache and only query specifi info you ight need. 

# Geographic information

Often it is interesting to see where IPs are coming from. A cool way to do this is using MaxMinds free [GeoLite2](https://dev.maxmind.com/geoip/geolite2-free-geolocation-data) data set and [folium](http://python-visualization.github.io/folium/), a python interface to [leaflet](https://leafletjs.com) allowing one to create interactive maps. By default folium/leaflet use openstreetmap.

MaxMind offers an API, but it is much slower than a local lookup, and it does not disclose the IPs you are looking at. 

While the code below may look complicated it's actually quite simple. We first define a function that retruns a color for a given value between 0 and 1. 

Then we set up the geoip and map objects. What we really want to plot though is the value counts of the ips. 

The lines
```
tmp = df['IP'].value_counts()
values = list(zip(tmp.index.tolist(), tmp.values.tolist() ) )
```

create a list of ip count pairs out of the pandas Series. The rest is really pretty straight forward: Loop over the entries and plot a colored dot at the coordinates of each IP. 


In [ ]:
import geoip2.database
import folium
import math
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib


#cmap = matplotlib.cm.get_cmap('YlOrRd')
cmap = matplotlib.colormaps['YlOrRd']
def get_color( v ):
    '''Return a palette RGB value for an argumennt between 0 and 1.'''
    (r, g, b) = [ int(255 * i) for i in  cmap(1-v)[:-1] ]
    #print( r,g,b )
    return '#{:06X}'.format( 256 * ( 256 * r + g ) + b )  

reader = geoip2.database.Reader( os.path.expanduser('data/GeoLite2-City_20240611/GeoLite2-City.mmdb') )
m = folium.Map(zoom_start=2, location=[0,0])
# 
tmp = df['IP'].value_counts()
values = list(zip(tmp.index.tolist(), tmp.values.tolist() ) )
m_value = values[0][1]

for i in values:
    try:
        response = reader.city( i[0] )
    except:
        continue
    c = get_color( math.log( i[1] ) / math.log( m_value) )
    pos = ( response.location.latitude , response.location.longitude )
    #folium.Marker( location=pos,icon=folium.Icon(color='darkpurple') ).add_to(m)
    try:
        folium.CircleMarker( pos , radius=2, color=c ).add_to(m)
    except Exception as e:
        print(e, i, pos)
m

The result, while looking cool is hard to read. It's much moch more interesting to combine, or cluster, points nearby. Folium can do that fo you, check it out.

In [ ]:
from folium.plugins import MarkerCluster


#cmap = matplotlib.cm.get_cmap('YlOrRd')
cmap = matplotlib.colormaps['YlOrRd']
def get_color( v ):
    '''Return a palette RGB value for an argumennt between 0 and 1.'''
    (r, g, b) = [ int(255 * i) for i in  cmap(1-v)[:-1] ]
    #print( r,g,b )
    return '#{:06X}'.format( 256 * ( 256 * r + g ) + b )  

reader = geoip2.database.Reader( os.path.expanduser('data/GeoLite2-City_20240611/GeoLite2-City.mmdb') )

tmp = df['IP'].value_counts()

values = list(zip(tmp.index.tolist(), tmp.values.tolist() ) )

m = folium.Map(zoom_start=2, location=[0,0])
m_value = values[0][1]

marker_cluster = MarkerCluster()


for i in values:
    try:
        response = reader.city( i[0] )
    except:
        continue
    c = get_color( math.log( i[1] ) / math.log( m_value) )
    pos = ( response.location.latitude , response.location.longitude )
    try:
        folium.CircleMarker( pos , radius=5, color=c, fill=True, fill_opacity=1 ).add_to(marker_cluster)
    except:
        print(i)

marker_cluster.add_to(m)
folium.LayerControl().add_to(m)
m

These maps are only viewable in a webbrowser. But you might need classic pixmaps.  This can be achived with a little workaround: Rendering the map in a virtual browser and taking a screenshot. Note, that this needs [selenium](https://selenium-python.readthedocs.io/) to be installed. The croping needs to be finetuned a bit by hand.

In [ ]:
import io
from PIL import Image
import os 
import logging

os.environ["LD_LIBRARY_PATH"] = "/opt/conda/lib/"
os.environ["SE_MANAGER_PATH"] = "/opt/conda/bin/selenium-manager"

def map_to_png( m ):
    img_data = m._to_png(5)
    # crop( ( left, upper, right, and lower ))
    return Image.open(io.BytesIO(img_data)).crop( (170,20, 1195,600))
    #img.save('image.png')
display(map_to_png(m))